In [1]:
!pip install pycaret


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.8 MB/s eta 0:00:00
   ━━━━

In [1]:
import numpy as np


data = np.load('attack_model_train_data.npz')
X = data['X']
Y = data['Y']

In [2]:
import pandas as pd

# Assume X_train and Y_train are your feature and target arrays
# Create a DataFrame for X_train and add Y_train as the target column
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df['target'] = Y


In [7]:
from pycaret.classification import *

In [9]:
from pycaret.classification import *

# Initialize PyCaret with the class imbalance correction
clf_setup = setup(data=df,
                  target='target',
                  fix_imbalance=True,   # Automatically applies SMOTE
                  session_id=42)        # For reproducibility


,Description,Value
0,Session id,42
1,Target,target
2,Target type,Binary
3,Original data shape,"(48000, 11)"
4,Transformed data shape,"(70368, 11)"
5,Transformed train set shape,"(55968, 11)"
6,Transformed test set shape,"(14400, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


In [13]:
# Compare models and get the best one based on AUC (or Recall if preferred)
best_model_auc = compare_models(sort='AUC')  # Use 'Recall' if that better suits your objective


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8062,0.7010,0.8843,0.8831,0.8837,0.3018,0.3018,18.1800
ada,Ada Boost Classifier,0.8033,0.6969,0.8806,0.8829,0.8818,0.2972,0.2972,3.7770
rf,Random Forest Classifier,0.7615,0.6652,0.8332,0.8746,0.8530,0.2183,0.2208,22.2330
lightgbm,Light Gradient Boosting Machine,0.7393,0.6591,0.7993,0.8768,0.8363,0.2038,0.2092,3.5000
et,Extra Trees Classifier,0.7542,0.6516,0.8274,0.8710,0.8486,0.1973,0.1992,10.1370
xgboost,Extreme Gradient Boosting,0.7361,0.6501,0.7974,0.8746,0.8342,0.1939,0.1990,1.1920
knn,K Neighbors Classifier,0.6569,0.6265,0.6864,0.8747,0.7692,0.1374,0.1540,1.8380
dt,Decision Tree Classifier,0.7120,0.5937,0.7791,0.8618,0.8180,0.1328,0.1368,1.5520
qda,Quadratic Discriminant Analysis,0.7381,0.5324,0.8608,0.8309,0.8442,-0.0141,-0.0149,0.2940
nb,Naive Bayes,0.4230,0.5315,0.3739,0.8479,0.5148,0.0208,0.0323,0.2780


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [25]:
best_model_auc = tune_model(best_model_auc)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:36:14
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


KeyboardInterrupt: 

In [26]:
X_attack = np.load('attack_test.npy')
Y_attack = np.load('attack_test_label.npy')

In [27]:
df_attack = pd.DataFrame(X_attack, columns=[f'feature_{i}' for i in range(X_attack.shape[1])])
df_attack['target'] = Y_attack

In [28]:
# Predict on the test set
predictions = predict_model(best_model_auc, data=df_attack)

# Evaluate the model's performance
evaluate_model(best_model_auc)

# Alternatively, for a more detailed evaluation
from sklearn.metrics import classification_report, accuracy_score

# Make predictions using the best model on test data (assuming the test data is in the same format)
y_pred = best_model_auc.predict(df_attack.drop(columns=['target']))
print("Accuracy:", accuracy_score(df_attack['target'], y_pred))
print("Classification Report:\n", classification_report(df_attack['target'], y_pred))


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.6866,0.6113,0.7371,0.8670,0.7968,0.1320,0.1407


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Accuracy: 0.68665
Classification Report:
               precision    recall  f1-score   support

           0       0.25      0.43      0.32     10000
           1       0.87      0.74      0.80     50000

    accuracy                           0.69     60000
   macro avg       0.56      0.59      0.56     60000
weighted avg       0.76      0.69      0.72     60000



In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
precision = precision_score(df_attack['target'], y_pred)
recall = recall_score(df_attack['target'], y_pred)
f1 = f1_score(df_attack['target'], y_pred)
accuracy = accuracy_score(df_attack['target'], y_pred)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 0.8669693476675527
Recall: 0.73708
F1 Score: 0.7967657200920991
Accuracy: 0.68665


In [30]:
from sklearn.metrics import confusion_matrix

print(classification_report(df_attack['target'], y_pred))

print(confusion_matrix(df_attack['target'], y_pred))

              precision    recall  f1-score   support

           0       0.25      0.43      0.32     10000
           1       0.87      0.74      0.80     50000

    accuracy                           0.69     60000
   macro avg       0.56      0.59      0.56     60000
weighted avg       0.76      0.69      0.72     60000

[[ 4345  5655]
 [13146 36854]]


In [13]:
save_model(best_model_auc, 'best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['feature_0', 'feature_1',
                                              'feature_2', 'feature_3',
                                              'feature_4', 'feature_5',
                                              'feature_6', 'feature_7',
                                              'feature_8', 'feature_9'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strat...
                          